# **Green Bean Price Tracker**

## Passing login form with Requests and BautifulSoup

Imports (it is successful if there are no errors).

In [ ]:
# Standart packages
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

# E-mail notifictions packages
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Custom imports
import config as cfg 

__Requests__ is an HTTP library for Python, that allows to send HTTP/1.1 requests easy (no need to manually add query strings to your URLs, or to form-encode your POST data.) Keep-alive and HTTP connection pooling are 100% automatic, thanks to urllib3.

__Beautiful Soup__ is a Python library for pulling data out of HTML and XML files (we will work with HTML files), by representing the HTML as a set of objects with methods used to parse the HTML. We can navigate the HTML as a tree and/or filter out what we are looking for.  

In [ ]:
# URL of our target page we gonna scrap.
url = "https://offerlist.rehmcoffee.de"

__Session__ object because:
+ it allows us to persist certain parameters across requests
+ it persists cookies across all requests made from the Session instance 
+ It has all the methods of the main Requests API

In [ ]:
s = requests.Session()

Firstly, we use __Requests__ to get access to the page content.

In [ ]:
# Using Requests's method get on session object to get access to the page content.
login_form = s.get(url)

In [ ]:
# Printing out the URL of the page to check if everything works proper.
login_form.url

In [ ]:
# Calling attribute content to see the content of the login page before we try to log in.
login_form.content

To parse a document, we pass it into the `BeautifulSoup` constructor.Then we create a `BeautifulSoup` object : *soup* , which represents the document as a nested data structure.

In [ ]:
soup_login_form = BeautifulSoup(login_form.content,"html5lib")

In [ ]:
# Using prettify() method  to display the HTML in the nested structure:
print(soup_login_form.prettify())

Next, we search for inputs are required to submit to login form to pass it. Below we can see that not _user_ and _password_ but also few extra tokens are required. 

In [ ]:
list_input = soup_login_form.find_all("input")
list_input

In [ ]:
token_nr1 = soup_login_form.find("input", {"name":"logintype"})["value"]
token_nr2 = soup_login_form.find("input", {"name":"pid"})["value"]
token_nr3 = soup_login_form.find("input", {"name":"redirect_url"})["value"]
token_nr4 = soup_login_form.find("input", {"name":"tx_felogin_pi1[noredirect]"})["value"]

In [ ]:
print(token_nr1)
print(token_nr2)
print(token_nr3)
print(token_nr4)

Earlier we created __config.py__ to store sensitive data as usernames and passwords. 

If you try to find register form at https://offerlist.rehmcoffee.de , there is none. You need to contact company and request login info. Be aware you will need to provide company info to get access to offer page. 


__config.py__ consists of a line of code, a dictionary of the following format:

`login_data = {'user': 'name@company-name.com', 'pass': 'password_received_from_trader'}`
A config file has to be created in the root directory of the project and it will be called from the code as follows:

In [ ]:
#login_data = {"user":"mymail@company-name.com", "pass":"your_password", "logintype":token_nr1, "pid":token_nr2, "redirect_url":token_nr3, "tx_felogin_pi1[noredirect]":token_nr4}
login_data = cfg.login_data
login_data["logintype"] = token_nr1
login_data["pid"] = token_nr2
login_data["redirect_url"] = token_nr3
login_data["tx_felogin_pi1[noredirect]"] = token_nr4

In [ ]:
s.post(url, login_data)

__Status code:200__ in the Response message in HTTP Protocol stays for OK (any 2xx stays for Success). If we skip `Session()` and go for `requests.get(url)` above, while our response code will be still OK we will not stay logged in what will result in landing on login form again again. 

In [ ]:
offer_page = s.get(url)

In [ ]:
offer_page.content

Going throgh content we see that we successfully passed the login form and have excess to data we going to scrap. 

## Using **BeautifulSoup** and **Pandas** to extract the data into DataFrame

Pass HTML into the `BeautifulSoup` constructor, then create a new `BeautifulSoup` object : *soup_offer_page* , which represents the HTML code as a nested data structure.

In [ ]:
soup_offer_page = BeautifulSoup(offer_page.content,"html5lib")

In [ ]:
print(soup_offer_page.prettify())

In [ ]:
# Checking how many tables are on page.

tables = soup_offer_page.find_all('table')
len(tables)

In [ ]:
tables

The output is 3 tables. If we visit the  https://offerlist.rehmcoffee.de/ we can see exactly 3 tables: Stock Exchange, Your Contact and the third one in which on fact we are interested in.

Now we automate table choice. Below the loop that search for words in tables and prints an index of a table in cell below, than displays the html code as nested structure of this table in the next cell. (we need to feed it with words unique to the table we are searching).

In [ ]:
for index,table in enumerate(tables):
    if ("almond" in str(table)):
        table_index = index
        
print(table_index)

In [ ]:
print(tables[table_index].prettify())

It is possible to scrape data from HTML tables into a DataFrame using BeautifulSoup and the Pandas function `read_html`that creates a DataFrame and populates it.

Our table is `tables[table_index]`. 

When we use the pandas function `read_html`, we give it the string version of the table as well as the flavor which is the parsing engine bs4.

The function `read_html` always returns a list of DataFrames so we must pick the one we want out of the list. We use `[0]` index as we already spicified proper table from tables above.

We can also use the read_html function to directly get DataFrames from a url and than pick the DataFrame we need out of the list as follows (but this works when we don't need to pass a login form):
<code>
whole_page_df = pd.read_html(url, flavor='bs4')
len(whole_page_df)
whole_page_df[1]
<code>


In [ ]:
offer_list_rehm_df = pd.read_html(str(tables[table_index]), flavor='bs4')[0]
offer_list_rehm_df

## Data preparation before storaging 

### Adding Date column

We create Timestamp object that provides current Timestamp, using Pandas `.to_datetime` method on arg 'today' to get current timestamp (not just date) in local timezone and `normalize()` to keep the date as a Timestamp.

In [ ]:
current_date = pd.to_datetime('today').normalize()
current_date

In [ ]:
offer_list_rehm_df['offer_date'] = current_date
offer_list_rehm_df

### Modifying Unit, € / KG and D / KG columns values

Copy dataframe to check how our functions acts and modify data only after all functions correct

In [ ]:
offer_list_rehm_df_copy = offer_list_rehm_df.copy(deep=True)

offer_list_rehm_df_copy

First we create functions to modify the column's values

In [ ]:
def clean_unit_col(x):
    x = x.replace("kg", "").replace(" ", "")
    return int(x)

def clean_eur_col(y):
    y = y.replace("€", "").replace(",", ".").replace(" ", "")
    return round(float(y),2)

def clean_usd_col(z):
    z = z.replace("$", "").replace(",", ".").replace(" ", "")
    return round(float(z),2)

Now we pass it to apply method. 

In [ ]:
offer_list_rehm_df_copy['Unit'] = offer_list_rehm_df_copy['Unit'].apply(clean_unit_col)

offer_list_rehm_df_copy['€ / KG'] = offer_list_rehm_df_copy['€ / KG'].apply(clean_eur_col)

offer_list_rehm_df_copy['$ / KG'] = offer_list_rehm_df_copy['$ / KG'].apply(clean_usd_col)

offer_list_rehm_df_copy.head(20)

In [ ]:
# IMPORTANT: run only first time!
offer_list_rehm_df_copy.to_csv('scraped-data/rehm-offer-list.csv',index=False)

Now our info bundle is completed. We can move further and place it in file. 

## Placing scraped data for storage in .csv file using  **Pandas** on local machine

Using cell magic we execute `bash` commands to create folders to place our scraped data in a properly arranged manner.

In [ ]:
%%bash
mkdir green-bean-price-tracker 
cd green-bean-price-tracker 
mkdir scraped-data
cd ~

Using line magic we check Path Working Directory to be sure that we got back to proper location.

In [ ]:
%pwd

Using `to_scv` method we create .csv file where we will collect our data as green bean trader publish new updates to offer list. 

In [ ]:
offer_list_rehm_df_copy.to_csv('scraped-data/rehm-offer-list.csv',index=False)

Now we can check our folders and the.csv file manually to be sure all in place. If we open .csv file with Excel we can see some formating issues therefore we will check the values calling `read_csv` to be sure all values are intact. If you got Error trying to read file, check whether you closed the fail in Excel after checking it. 

In [ ]:
pd.read_csv('scraped-data/rehm-offer-list.csv')

## Reparing .csv fail



In [ ]:
df_read_forteen = pd.read_csv('scraped-data/rehm-offer-list-14.04.2021.csv')

In [ ]:
df_read_forteen.tail()

In [ ]:
df_read_twentyone.to_csv('scraped-data/rehm-offer-list-new.csv',index=False)

In [ ]:
df_read_twentyeight = pd.read_csv('scraped-data/rehm-offer-list-28.04.2021.csv', sep=',' ,  index_col=False)
df_read_twentyeight.head()

In [ ]:
df_read_the_last = pd.read_csv('scraped-data/rehm-offer-list.csv')
df_read_the_last.tail()

In [ ]:
df_read_the_last.to_csv('scraped-data/rehm-offer-list-new.csv',mode='a',index=False,header=False)

In [ ]:
df_read_today = pd.read_csv('scraped-data/rehm-offer-list-new.csv')
df_read_today.head(214)

In [ ]:
offer_list_rehm_df_copy.to_csv('scraped-data/rehm-offer-list-new.csv',mode='a',index=False,header=False)
#offer_list_rehm.to_csv('scraped-data/rehm-offer-list-new.csv',mode='a',index=False,header=False)

## Appending .csv fail with new data release

The green bean trader publishes updates to the offer list weekly. We plan to log in weekly and append the .csv fail with full list. 

In [ ]:
# IMPORTANT: run only when new data is available from trader!
offer_list_rehm.to_csv('scraped-data/rehm-offer-list.csv',mode='a',index=False,header=False)

Checking whether new data is available from trader.

In [ ]:
df_today = pd.read_csv('scraped-data/rehm-offer-list.csv')
df_previous = pd.read_csv('scraped-data/rehm-offer-list-14.04.2021.csv')

In [ ]:
df_previous.Bags == offer_list_rehm.Bags 

In [ ]:
df_previous.Bags.compare(offer_list_rehm.Bags)

## Compare dataframes

In [ ]:
df_last = pd.read_csv('scraped-data/rehm-offer-list.csv', index_col = False)
#df_last = df_last.reset_index(drop=True)
df_last

In [ ]:
df_twentyone = pd.read_csv('scraped-data/rehm-offer-list-21.04.2021.csv')

In [ ]:
df_twentyone

In [ ]:
df_twentyone.set_index(['Farm / Name'])

In [ ]:
df_twentyone.set_index(['Farm / Name']).sort_index(ascending=True).loc['Burka Estate AB']

In [ ]:
df_last.compare(df_twentyone, keep_shape=True)

## Data Visualization


In [ ]:
df_read = pd.read_csv('scraped-data/rehm-offer-list.csv')

In [ ]:
df_likelast = df_read.loc[df_read['offer_date'] == current_date]

In [ ]:
df_likelast

In [ ]:
df_last.describe()

For the "Bags" provide a boxplot

In [ ]:
ax = sns.boxplot(x='Bags', data = df_last)

Create a histogram for the "Bags" variable

In [ ]:
ax = sns.displot(df_last['Bags'], kde = True)

For the "€ / KG" provide a boxplot

In [ ]:
ax = sns.boxplot(x='€ / KG', data = df_last)

Provide a boxplot for the "€ / KG" variable vs the "Bags" variable. (Discretize the "€ / KG" variable into three groups of 4 € / KG and less, between 4 and 8 € / KG and 8 € / KG and higher)

In [ ]:
df_last.loc[(df_last['€ / KG'] <= 4), 'price_group'] = '4 EUR and less'
df_last.loc[(df_last['€ / KG'] > 4)&(df_last['€ / KG'] < 8), 'price_group'] = 'between 4 and 8 EUR'
df_last.loc[(df_last['€ / KG'] >= 8), 'price_group'] = '8 EUR and up'

ax = sns.boxplot(x="price_group", y="Bags", data=df_last)

In [ ]:
sns.displot(x="total_KG_per_coffee",hue="price_group",multiple="stack",data=df_last)

In [ ]:
sns.displot(x="total_KG_per_coffee",hue="Origin", kind='kde',data=df_last)

Provide a scatter plot to show the relationship between "€ / KG" and the "Bags" per coffee. What is up with the relationship?

In [ ]:
ax = sns.scatterplot(x='€ / KG', y='Bags',  data=df_last)

In [ ]:
df_last['total_KG_per_coffee'] = df_last['Bags'] * df_last['Unit']

In [ ]:
df_last['total_EUR_per_coffee'] = df_last['total_KG_per_coffee'] * df_last['€ / KG']

In [ ]:
df_last.head(5)

In [ ]:
sns.histplot(df_last[df_last['Origin'] == 'BRAZIL']['total_KG_per_coffee'], color='green', kde=False) 
sns.histplot(df_last[df_last['Origin'] == 'PANAMA']['total_KG_per_coffee'], color="blue", kde=False) 
plt.show()

In [ ]:
country_summary = df_last.groupby('Origin')['total_EUR_per_coffee'].sum()

In [ ]:
type(country_summary)

In [ ]:
country_summary

In [ ]:
country_summary.sum()

## E-mail notifications

In [ ]:
email_body = "<html><body>Hey folks! That's green-bean-price-tracker bot here. I've prepared some fresh tasty updates to coffee offers by Rehm from: "

email_body += str(current_date)

email_body += '<br>actual info will be inserted here'

email_body += '<br>Yours green-bean-price-tracker bot</body></html>'

In [ ]:
# Email Account
email_sender_account = cfg.email_sender_account # email_sender_account = "your.registered.name@gmail.com"
email_sender_username = cfg.email_sender_username # email_sender_username = "your.registered.name"
email_sender_password = cfg.email_sender_password # email_sender_password = "your_gmail_password"
email_smtp_server = "smtp.gmail.com" # SMTP, eg smtp.gmail.com for gmail
email_smtp_port = 587 # SMTP Porf, eg 587 for gmail

# Email Content
email_recepients = ["yana.dzhulay@dbf-regensburg.de","mail@dbf-regensburg.de"]
email_subject = "test notification - weekly updates - green-bean-price-tracker "
#email_body = "here is the body"

In [ ]:
# login to email server
server = smtplib.SMTP(email_smtp_server,email_smtp_port)
server.starttls()
server.login(email_sender_username, email_sender_password)

# For loop, sending emails to all email recipients
for recipient in email_recepients:
    print("Sending email to: " , recipient)
    message = MIMEMultipart('alternative')
    message['From'] = email_sender_account
    message['To'] = recipient
    message['Subject'] = email_subject
    message.attach(MIMEText(email_body, 'html'))
    text = message.as_string()
    server.sendmail(email_sender_account,recipient,text)
    
# All emails sent, log out.
server.quit()